### NN for Connect 4


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

Extracting data

In [3]:
# fetch dataset 
connect_4 = fetch_ucirepo(id=26) 
# metadata 
print(connect_4.metadata)  
# variable information 
print(connect_4.variables) 

{'uci_id': 26, 'name': 'Connect-4', 'repository_url': 'https://archive.ics.uci.edu/dataset/26/connect+4', 'data_url': 'https://archive.ics.uci.edu/static/public/26/data.csv', 'abstract': 'Contains connect-4 positions', 'area': 'Games', 'tasks': ['Classification'], 'characteristics': ['Multivariate', 'Spatial'], 'num_instances': 67557, 'num_features': 42, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1995, 'last_updated': 'Sat Mar 09 2024', 'dataset_doi': '10.24432/C59P43', 'creators': ['John Tromp'], 'intro_paper': None, 'additional_info': {'summary': 'This database contains all legal 8-ply positions in the game of connect-4 in which neither player has won yet, and in which the next move is not forced.\r\n\r\nx is the first player; o the second.\r\n\r\nThe outcome class is the game theoretical value for the first player.', 'purpose': None, 'funded_b

Getting X and y

In [4]:
# data (as pandas dataframes) 
X = connect_4.data.features 
y = connect_4.data.targets 
#intepret connect 4 dataset
print(X.shape)
print(y.shape)
# print(X.iloc[0,:])
# print(y.iloc[:])

(67557, 42)
(67557, 1)


Label Encoding X and y 
This is to helping with normalizing data and making it easier to work with


In [50]:
#Since x data are categorical, we need to encode them

#X -> x = 1 , o = -1 , b = 0
X_r = X.replace({'x':1,'o':2,'b':0})
#y -> win = 1 , draw = 0, loss = -1
y_r = y.replace({'win':2,'draw':1,'loss':0})
print(y_r.shape)
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_r, y_r, test_size=0.2, random_state=42)
# Convert pandas dataframes to PyTorch tensors, do not use dataloader
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long).squeeze()
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.long).squeeze()



(67557, 1)


C:\Users\johna\AppData\Local\Temp\ipykernel_25976\3232681351.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_r = X.replace({'x':1,'o':2,'b':0})
C:\Users\johna\AppData\Local\Temp\ipykernel_25976\3232681351.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_r = y.replace({'win':2,'draw':1,'loss':0})


Using SKlearns random forest classifier to train the model

In [35]:
# Train a Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Test the model
y_pred = clf.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred):.4f}')


Accuracy: 0.8203


Using SKlearn logistic regression to train the model

In [36]:
#try with different models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Train a Logistic Regression model
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred):.4f}')


Accuracy: 0.6576


Using SKlearn MLP classifier

In [37]:
#use NN

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred):.4f}')



Accuracy: 0.8134


c:\Users\johna\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Nn for connect 4

In [67]:
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define a simple neural network
class Connect4NN(nn.Module):
    def __init__(self):
        super(Connect4NN, self).__init__()
        self.fc1 = nn.Linear(42, 128)
        self.droupout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 3)
        self.relu = nn.ReLU() #rmb cross entropy already has softmax
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

    
model = Connect4NN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
model.train()
for epoch in range(100):
    runningloss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        runningloss += loss.item()
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, loss: {runningloss/len(train_loader)}')
    
# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        _, predicted = torch.max(y_pred, 1)
        total += y_batch.size(0) #0 is the number of elements in the tensor
        correct += (predicted == y_batch).sum().item()
print(f'Accuracy: {correct/total:.4f}')

Epoch 10, loss: 0.4500786323919364
Epoch 20, loss: 0.3916770363882642
Epoch 30, loss: 0.35550200657359815
Epoch 40, loss: 0.33053254295607265
Epoch 50, loss: 0.3142285339939397
Epoch 60, loss: 0.296659553128495
Epoch 70, loss: 0.279640196027767
Epoch 80, loss: 0.2660755416462044
Epoch 90, loss: 0.25703442484210853
Epoch 100, loss: 0.24564706749033985
Accuracy: 0.8188


In [39]:
#save model weights as numpy array
model_weights = model.state_dict()
for key in model_weights:
    model_weights[key] = model_weights[key].numpy()
print(model_weights)

#i want to save the model weights and copy is direcly 


OrderedDict({'fc1.weight': array([[-0.9172921 ,  0.01317978,  0.04196112, ...,  0.10881053,
        -0.18134035,  0.02382498],
       [-0.11771316, -0.09623528, -0.07955053, ...,  0.04455304,
        -0.2419225 ,  0.05795357],
       [-0.08505697, -0.20643847,  0.2649403 , ..., -0.22551998,
         0.33757955,  0.05375292],
       ...,
       [-0.1371001 , -0.12976906,  0.02088212, ..., -0.13445982,
        -0.1397467 , -0.03763557],
       [ 0.01979042, -0.09754016,  0.03448568, ...,  0.07651879,
         0.11732315,  0.02098016],
       [-0.05844294, -0.0510306 , -0.41637105, ...,  0.03920374,
         0.37483162, -0.03420401]], dtype=float32), 'fc1.bias': array([-0.01853321, -0.05241129,  0.10026716, -0.12027285,  0.0133744 ,
       -0.1389007 , -0.05916514, -0.03272083, -0.13664131,  0.08095073,
       -0.09348211, -0.15980193, -0.125313  , -0.16184181, -0.1466474 ,
       -0.11890833,  0.0019084 , -0.14425734,  0.04845731,  0.13099606,
        0.04800085, -0.08505814, -0.00918544

In [42]:
#save model weights as numpy array in a file
import numpy as np
np.save('model_weights.npy', model_weights)


In [49]:
#test on first y_test
model.eval()
print(y_test[1])
with torch.no_grad():
    y_pred = model(X_test[1])
    _, predicted = torch.max(y_pred, 0)
    print(predicted.item())
    
    
    
    


tensor(2)
2
